In [1]:
import pandas as pd
import glob
import numpy as np
colunas_necessarias = ['CPF', 'NOME', '01_DDDFONE', '02_DDDFONE','03_DDDFONE','EMAIL1', 'EMAIL2','01_BAIRRO', '01_CIDADE', '01_UF', '01_CEP']
# Varrendo a pasta com os arquivos da intersic, tudo que tiver TXT vai ser aberto em DF_NET
caminho = r"R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV"
arquivos = glob.glob(caminho + "/*.csv")

lista = []
for filename in arquivos:
    higi_afinder = pd.read_csv(filename, index_col=None, header=0, encoding='iso-8859-1',sep=";",dtype=str,usecols=colunas_necessarias)
    print(filename)
    lista.append(higi_afinder)


higi_afinder = pd.concat(lista, ignore_index=True)

# TRATANDO CPF E CNPJ, INCLUINDO ZEROS A ESQUERDA.
df_cpfs_correto = higi_afinder[higi_afinder['CPF'].str.len()==11]
df_cnpjs_correto = higi_afinder[higi_afinder['CPF'].str.len()==14]
df_cpfs_corrigir = higi_afinder[higi_afinder['CPF'].str.len()<11]
df_cpfs_corrigir['CPF'] = df_cpfs_corrigir['CPF'].apply(lambda x: '{0:0>11}'.format(x))
df_cnpj_corrigir = higi_afinder[higi_afinder['CPF'].str.len()>11]
df_cnpj_corrigir['CPF']  = df_cnpj_corrigir['CPF'].apply(lambda x: '{0:0>14}'.format(x))
higi_afinder = pd.concat([df_cpfs_correto,df_cpfs_correto,df_cpfs_corrigir,df_cnpj_corrigir],ignore_index=True)\
.drop_duplicates(['CPF','NOME','01_DDDFONE','02_DDDFONE','03_DDDFONE','EMAIL1','EMAIL2'])


#Criando uma coluna email coringa pra gerar o layout necessário um embaixo do outro
higi_afinder = higi_afinder.assign(EMAIL_3='')

higi_afinder.rename(columns={'01_DDDFONE': 'TELEFONE'}, inplace=True)
higi_afinder.rename(columns={'02_DDDFONE': 'TELEFONE'}, inplace=True)
higi_afinder.rename(columns={'03_DDDFONE': 'TELEFONE'}, inplace=True)
higi_afinder.rename(columns={'EMAIL1': 'EMAIL'}, inplace=True)
higi_afinder.rename(columns={'EMAIL2': 'EMAIL'}, inplace=True)
higi_afinder.rename(columns={'EMAIL_3': 'EMAIL'}, inplace=True)


# SEPARANDO 3 DF'S PARA COLOCAR OS TELEFONES / EMAILS UM EMBAIXO DO OUTRO

df_1 = higi_afinder.iloc[:,[0,1,2,5,7,8,9,10]]
df_2 = higi_afinder.iloc[:,[0,1,3,6,7,8,9,10]]
df_3 = higi_afinder.iloc[:,[0,1,4,11,7,8,9,10]]
# CONCATENANDO OS DF'S EM UM
higi_afinder = pd.concat([df_1,df_2,df_3])


# SUBSTITUINDO NAN POR 0 EM TELEFONE

higi_afinder['TELEFONE'] = higi_afinder['TELEFONE'].fillna('')

#SUBSTITUINDO NAN POR 'VAZIO' EM  EMAIL.

higi_afinder['EMAIL'] = higi_afinder['EMAIL'].fillna('')

# #Contando os caracteres na coluna Telefone E TRAZENDO TUDO QUE FOR > 9, ISSO DELETA OS ZEROS*
# #Organizando a coluna CPF por ordem PARA APARECER O MESMO CLIENTE, LINHA A LINHA DEPENDENDO DA QUANTIDADE DE DADOS ELE TIVER
higi_afinder = higi_afinder.sort_values(by=['CPF'],ignore_index=True)

# Separando arquivo no layout do banco provisoriamente.
higi_afinder_layout_banco = higi_afinder

#Filtrando cpf / nome / tel (sem vazio no tel)
df_telefone_higi = higi_afinder.iloc[:,[0,1,2]]\
.query('TELEFONE != ""')
df_telefone_higi

df_telefone_higi_1 = df_telefone_higi[0:1000000]
df_telefone_higi_2 = df_telefone_higi[1000001:2000000]


#Filtrando cpf / nome / email (sem vazio no email)
df_email_higi = higi_afinder.iloc[:,[0,1,3]]\
.query('EMAIL != ""')



### FILTRANDO SOMENTE AS LINHAS QUE DE FATO TEM TELEFONE E EMAIL, DESCONSIDERANDO OS VAZIOS.
df_telefone_higi = df_telefone_higi.query("TELEFONE != ' '")
df_email_higi = df_email_higi.query("EMAIL != ' '")

display(df_telefone_higi)
display(df_email_higi)

R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV\afinder-enriquecimento_OK_0_CLAR_NET_GEV_lote_0_COBRANCA_.csv
R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV\afinder-enriquecimento_OK_0__CLAR_NET_GEV_lote_1_COBRANCA_.csv
R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV\afinder-enriquecimento_OK_0__CLAR_NET_GEV_lote_2_COBRANCA_.csv


C:\Users\jorgean.bomfim\AppData\Local\Temp\ipykernel_16772\3846214845.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cnpj_corrigir['CPF']  = df_cnpj_corrigir['CPF'].apply(lambda x: '{0:0>14}'.format(x))


,CPF,NOME,TELEFONE
0,00000033367,FRANCISCA MARLY FELIX SILVA,92991345324
1,00000033367,FRANCISCA MARLY FELIX SILVA,92993241623
3,00000059161,NATHALIA MOREIRA DE FIGUEIREDO,21972379242
6,00000070130,ANTONIA APARECIDA DE MATOS,11946455961
9,00000130567,IRACI DA SILVA SANTOS,99998211608
...,...,...,...
1970284,99999323049,TEREZINHA DE JESUS FRIGO,51995712236
1970285,99999323049,TEREZINHA DE JESUS FRIGO,51999310430
1970286,99999579353,FRANCISCA MARIA RODRIGUES DA SILVA,11954100455
1970288,99999579353,FRANCISCA MARIA RODRIGUES DA SILVA,11972103741


,CPF,NOME,EMAIL
3,00000059161,NATHALIA MOREIRA DE FIGUEIREDO,nathaliafigueiredo9@hotmail.com
21,00000402583,JOAO GOMES DA SILVA,joao.silva@cesp.com.br
23,00000402583,JOAO GOMES DA SILVA,renatogomesbt@terra.com.br
57,00000938157,ADRIANA SANTOS DE CASTRO,adrianasantosdecastro@gmail.com
59,00000938157,ADRIANA SANTOS DE CASTRO,adriana.s.castro@hotmail.com
...,...,...,...
1970238,99994828568,EDUARDO MENDES DA SILVA,duardomendes@gmail.com
1970248,99995786834,FLORIAL ALVES,florial.alves@bol.com.br
1970250,99995859815,LIGIA DOS SANTOS CAMPOS,ligiascampos@gmail.com
1970251,99995859815,LIGIA DOS SANTOS CAMPOS,campos.ligia_santos@hotmail.com


In [5]:
df_email_higi.to_csv(r"R:\TI\TELEFONIA\BASES_HIGIENIZADAS_CLARO_E_NET\EMAILS CLARO E NET/HIGIENIZACAO_EMAIL_NET_DIA_0208_NAO_IMPORTADA.csv",index=False,sep=';')
df_telefone_higi.to_csv(r"R:\TI\TELEFONIA\BASES_HIGIENIZADAS_CLARO_E_NET\TELEFONES CLARO E NET/HIGIENIZACAO_NET_DIA_0208_NAO_IMPORTADA.csv",index=False,sep=';')

### SEPARANDO A BASE VALIDADA

In [6]:
### LENDO A BASE QUE VOLTOU DA VALIDACAO CARLIM
import pandas as pd 
import numpy as np
import glob  
import warnings
import datetime

warnings.filterwarnings('ignore')
colunas_necessarias = ['CPF','01_DDDFONE','01_ZAP','02_DDDFONE','02_ZAP','03_DDDFONE','03_ZAP']
pd.set_option("display.max_columns", None)

caminho = r"R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV"
arquivos = glob.glob(caminho + "/*.csv")
#Lista de larguras do layout NET COB
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_csv(item, encoding='iso-8859-1', chunksize=500000, sep=";", dtype='str',usecols=colunas_necessarias)
        for df in chunks:
            dfList.append(df)
            
###########################################################################################################################################    

# Criando DataFrame com concat e a lista criada a cima
retorno_validado = pd.concat(dfList, sort=False, ignore_index=False)

# TRATANDO CPF E CNPJ, INCLUINDO ZEROS A ESQUERDA.
df_cpfs_correto = retorno_validado[retorno_validado['CPF'].str.len()==11]
df_cnpjs_correto = retorno_validado[retorno_validado['CPF'].str.len()==14]
df_cpfs_corrigir = retorno_validado[retorno_validado['CPF'].str.len()<11]
df_cpfs_corrigir['CPF'] = df_cpfs_corrigir['CPF'].apply(lambda x: '{0:0>11}'.format(x))
df_cnpj_corrigir = retorno_validado[retorno_validado['CPF'].str.len()>11]
df_cnpj_corrigir['CPF']  = df_cnpj_corrigir['CPF'].apply(lambda x: '{0:0>14}'.format(x))
retorno_validado = pd.concat([df_cpfs_correto,df_cpfs_correto,df_cpfs_corrigir,df_cnpj_corrigir],ignore_index=True)\
.drop_duplicates(['CPF', '01_DDDFONE', '01_ZAP', '02_DDDFONE', '02_ZAP', '03_DDDFONE','03_ZAP'])
retorno_validado = retorno_validado.reset_index(drop=True)

df_1 = retorno_validado.loc[:,['CPF','01_DDDFONE','01_ZAP']]
df_2 = retorno_validado.loc[:,['CPF','02_DDDFONE','02_ZAP']]
df_3 = retorno_validado.loc[:,['CPF','03_DDDFONE','03_ZAP']]

df_1 = df_1.rename(columns={"01_DDDFONE": 'Numero'})
df_2 = df_2.rename(columns={"02_DDDFONE": 'Numero'})
df_3 = df_3.rename(columns={"03_DDDFONE": 'Numero'})

df_1 = df_1.rename(columns={"01_ZAP": 'Usa_whatsapp'})
df_2 = df_2.rename(columns={"02_ZAP": 'Usa_whatsapp'})
df_3 = df_3.rename(columns={"03_ZAP": 'Usa_whatsapp'})

retorno_validado = pd.concat([df_1,df_2,df_3])

# Filtrando somente os que tem validação sim ou nao
retorno_validado = retorno_validado.query("Numero.notnull() and Usa_whatsapp.notnull() ")
retorno_validado['Usa_whatsapp'] = retorno_validado['Usa_whatsapp'].apply(lambda x: x.replace('1','Sim')
                                          .replace('0','Nao')
                                          .replace('2','Nao'))  

retorno_validado = retorno_validado.rename(columns={"Usa_whatsapp": 'Usa whatsapp'})
retorno_validado = retorno_validado.rename(columns={"Usa_whatsapp": 'Usa whatsapp'})
retorno_validado = retorno_validado.rename(columns={"Usa_whatsapp": 'Usa whatsapp'})



# Criando coluna com a data em que o código é rodado
data_atual = datetime.date.today()
data_envio = data_atual.strftime('%d/%m/%Y')
retorno_validado['data de envio'] = data_envio

retorno_validado = retorno_validado.assign(OBS="PITAO")
retorno_validado = retorno_validado.assign(Operadora="AFINDER")


# Colocando no layout e baixando a base
retorno_validado = retorno_validado.loc[:,['Numero','Operadora','Usa whatsapp','data de envio','OBS']]

### FILTRANDO APENAS OS TELEFONES QUE DE FATO TEM VALIDADOS SIM OU NAO
retorno_validado = retorno_validado.query("Numero != ' ' and   `Usa whatsapp`	!= ' '")

retorno_validado

R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV\afinder-enriquecimento_OK_0_CLAR_NET_GEV_lote_0_COBRANCA_.csv
R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV\afinder-enriquecimento_OK_0__CLAR_NET_GEV_lote_1_COBRANCA_.csv
R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\2\Jorgin da Clock\RETORNO HIGI CLARO E NET E GEV\afinder-enriquecimento_OK_0__CLAR_NET_GEV_lote_2_COBRANCA_.csv


,Numero,Operadora,Usa whatsapp,data de envio,OBS
0,11947006770,AFINDER,Sim,02/08/2023,PITAO
1,85998710472,AFINDER,Nao,02/08/2023,PITAO
2,89994061120,AFINDER,Sim,02/08/2023,PITAO
3,21999232507,AFINDER,Sim,02/08/2023,PITAO
4,53991750419,AFINDER,Nao,02/08/2023,PITAO
...,...,...,...,...,...
636486,55991787667,AFINDER,Nao,02/08/2023,PITAO
636499,53991305161,AFINDER,Nao,02/08/2023,PITAO
636504,55981162442,AFINDER,Nao,02/08/2023,PITAO
636505,55999203527,AFINDER,Nao,02/08/2023,PITAO


In [7]:
###### BAIXANDO 
retorno_validado.to_csv(r"R:\TI\TELEFONIA\BASE DE VALIDADOS/validados_afinder_net_gevenue_0208.csv",index=False, sep=';')